# 🎯 IELTS Momentum Engine
### A Streamlit Prototype for Personalised IELTS Preparation

**Steps to run this notebook:**
1. Run **Cell 1** — Install dependencies
2. Run **Cell 2** — Write the Streamlit app to a file
3. Run **Cell 3** — Launch the app and get a public URL via ngrok
4. Click the ngrok URL to open the live app in your browser

In [ ]:
# ============================================================
# CELL 1 — Install Required Packages
# ============================================================
!pip install -q streamlit pyngrok

In [ ]:
# ============================================================
# CELL 2 — Write the Streamlit App to app.py
# ============================================================
app_code = '''
import streamlit as st
import math

# ── Page Configuration ──────────────────────────────────────
st.set_page_config(
    page_title="IELTS Momentum Engine",
    page_icon="🎯",
    layout="centered"
)

# ── Custom CSS ───────────────────────────────────────────────
st.markdown("""
<style>
    html, body, [class*="css"]  {
        font-family: \'Segoe UI\', sans-serif;
    }
    .main-title {
        color: #1a4f8a;
        font-size: 2.4rem;
        font-weight: 800;
        text-align: center;
        margin-bottom: 0.2rem;
    }
    .subtitle {
        color: #4a7ab5;
        font-size: 1.05rem;
        text-align: center;
        margin-bottom: 1.8rem;
    }
    .section-header {
        color: #1a4f8a;
        font-size: 1.25rem;
        font-weight: 700;
        border-left: 5px solid #4a7ab5;
        padding-left: 10px;
        margin-top: 1.5rem;
        margin-bottom: 0.8rem;
    }
    .task-card {
        background: #f0f6ff;
        border-radius: 10px;
        padding: 14px 18px;
        margin-bottom: 10px;
        border: 1px solid #c8ddf5;
    }
    .metric-box {
        background: #1a4f8a;
        color: white;
        border-radius: 12px;
        padding: 18px;
        text-align: center;
        margin: 8px 0;
    }
    .metric-value {
        font-size: 2rem;
        font-weight: 800;
    }
    .metric-label {
        font-size: 0.85rem;
        opacity: 0.85;
        margin-top: 4px;
    }
    .on-track {
        background: #d4edda;
        border: 1px solid #28a745;
        color: #155724;
        border-radius: 10px;
        padding: 14px 18px;
        font-size: 1rem;
        font-weight: 600;
        text-align: center;
        margin-top: 12px;
    }
    .off-track {
        background: #fff3cd;
        border: 1px solid #ffc107;
        color: #856404;
        border-radius: 10px;
        padding: 14px 18px;
        font-size: 1rem;
        font-weight: 600;
        text-align: center;
        margin-top: 12px;
    }
    .danger {
        background: #f8d7da;
        border: 1px solid #dc3545;
        color: #721c24;
        border-radius: 10px;
        padding: 14px 18px;
        font-size: 1rem;
        font-weight: 600;
        text-align: center;
        margin-top: 12px;
    }
    div[data-testid="stCheckbox"] label {
        font-size: 0.97rem;
    }
</style>
""", unsafe_allow_html=True)

# ── Header ───────────────────────────────────────────────────
st.markdown(\'<div class="main-title">🎯 IELTS Momentum Engine</div>\', unsafe_allow_html=True)
st.markdown(\'<div class="subtitle">Your personalised daily IELTS preparation planner</div>\', unsafe_allow_html=True)

# ── Sidebar — User Inputs ────────────────────────────────────
with st.sidebar:
    st.image("https://upload.wikimedia.org/wikipedia/commons/f/ff/IELTS_logo.svg", width=130)
    st.markdown("### ⚙️ Your Profile")

    current_band = st.slider(
        "📊 Current Band Score",
        min_value=1.0, max_value=9.0, value=5.5, step=0.5,
        help="Your most recent or estimated IELTS Band Score"
    )
    target_band = st.slider(
        "🏆 Target Band Score",
        min_value=1.0, max_value=9.0, value=7.0, step=0.5,
        help="The IELTS Band Score you want to achieve"
    )
    days_left = st.number_input(
        "📅 Days Left for Exam",
        min_value=1, max_value=365, value=30, step=1,
        help="How many days until your IELTS exam?"
    )
    daily_hours = st.slider(
        "⏱️ Daily Preparation Time (hours)",
        min_value=0.5, max_value=8.0, value=2.0, step=0.5,
        help="How many hours can you dedicate per day?"
    )

    st.markdown("---")
    gap = round(target_band - current_band, 1)
    if gap <= 0:
        st.success("🎉 You have already reached your target band!")
    else:
        st.info(f"📌 Band Gap to Close: **{gap}**")

# ── Validate input ───────────────────────────────────────────
gap = round(target_band - current_band, 1)

# ── Study Plan Generator ─────────────────────────────────────
def generate_plan(daily_hours, current_band):
    \'\'\'Returns a list of (skill, task_description, weight) tuples.\'\'\'\'
    # Proportional time allocation (weights sum to 1)
    if current_band < 5.0:
        weights = {"Listening": 0.30, "Reading": 0.30, "Writing": 0.25, "Speaking": 0.15}
    elif current_band < 6.5:
        weights = {"Listening": 0.25, "Reading": 0.25, "Writing": 0.30, "Speaking": 0.20}
    else:
        weights = {"Listening": 0.20, "Reading": 0.20, "Writing": 0.35, "Speaking": 0.25}

    tasks = [
        {
            "skill": "🎧 Listening",
            "icon": "🎧",
            "minutes": round(weights["Listening"] * daily_hours * 60),
            "description": "Complete 1 full IELTS Listening section (Sections 1-2). Focus on note completion and multiple-choice questions. Review all incorrect answers.",
            "tip": "💡 Tip: Use official Cambridge IELTS practice tests for authentic difficulty.",
            "weight": weights["Listening"]
        },
        {
            "skill": "📖 Reading",
            "icon": "📖",
            "minutes": round(weights["Reading"] * daily_hours * 60),
            "description": "Read 1 academic passage and answer True/False/Not Given questions. Practise skimming and scanning techniques.",
            "tip": "💡 Tip: Time yourself strictly — 20 minutes per passage in the real exam.",
            "weight": weights["Reading"]
        },
        {
            "skill": "✍️ Writing",
            "icon": "✍️",
            "minutes": round(weights["Writing"] * daily_hours * 60),
            "description": "Write a Task 2 essay (250 words minimum) on a given topic. Focus on coherence, cohesion, and vocabulary range. Self-evaluate using the IELTS band descriptors.",
            "tip": "💡 Tip: Use linking words and varied sentence structures to improve your Coherence & Cohesion score.",
            "weight": weights["Writing"]
        },
        {
            "skill": "🗣️ Speaking",
            "icon": "🗣️",
            "minutes": round(weights["Speaking"] * daily_hours * 60),
            "description": "Record yourself answering Part 2 Speaking cue card questions. Listen back and note filler words, grammar errors, and pronunciation issues.",
            "tip": "💡 Tip: Practice with a timer — Part 2 requires 1-2 minutes of uninterrupted speaking.",
            "weight": weights["Speaking"]
        },
    ]
    return tasks

tasks = generate_plan(daily_hours, current_band)

# ── Daily Study Plan Section ─────────────────────────────────
st.markdown(\'<div class="section-header">📋 Today\'s Study Plan</div>\', unsafe_allow_html=True)
st.markdown(f"Based on **{daily_hours} hours/day** and your current band of **{current_band}**, here is your personalised plan:")

completed = []
for i, task in enumerate(tasks):
    with st.container():
        st.markdown(
            f\'\'\'<div class="task-card">
            <strong style="font-size:1.05rem;">{task["skill"]} &nbsp;·&nbsp; ⏱ {task["minutes"]} min</strong><br>
            <span style="color:#333;">{task["description"]}</span><br>
            <span style="color:#4a7ab5; font-size:0.9rem;">{task["tip"]}</span>
            </div>\'\'\',
            unsafe_allow_html=True
        )
        done = st.checkbox(f"Mark {task[\'skill\']} as completed", key=f"task_{i}")
        completed.append(done)

# ── Score Improvement Logic ──────────────────────────────────
def calculate_improvement(completed, tasks, gap, days_left, daily_hours):
    \'\'\'\'
    Estimate the predicted band score improvement based on:
    - Weighted task completion
    - Time available (days x hours)
    - Band gap to close
    \'\'\'\'
    if not any(completed):
        return 0.0, 0.0

    completion_weight = sum(
        task["weight"] for task, done in zip(tasks, completed) if done
    )

    # Max achievable improvement scales with days and hours
    total_hours = days_left * daily_hours
    # Diminishing returns: more hours help but with a log cap
    effort_factor = min(1.0, math.log1p(total_hours) / math.log1p(300))

    # Predicted daily improvement (fraction of gap)
    daily_improvement = completion_weight * 0.5 * effort_factor * gap

    # Over all remaining days
    total_predicted = round(min(daily_improvement * days_left, gap), 1)

    return total_predicted, completion_weight

predicted_improvement, completion_weight = calculate_improvement(
    completed, tasks, gap, days_left, daily_hours
)

predicted_score = round(min(current_band + predicted_improvement, 9.0), 1)
progress_pct = min(int((predicted_improvement / gap * 100) if gap > 0 else 100), 100)
tasks_done = sum(completed)
tasks_total = len(tasks)

# ── Results Section ──────────────────────────────────────────
st.markdown(\'<div class="section-header">📈 Your Progress Dashboard</div>\', unsafe_allow_html=True)

col1, col2, col3 = st.columns(3)
with col1:
    st.markdown(
        f\'\'\'<div class="metric-box">
        <div class="metric-value">+{predicted_improvement}</div>
        <div class="metric-label">Predicted Band Improvement</div>
        </div>\'\'\',
        unsafe_allow_html=True
    )
with col2:
    st.markdown(
        f\'\'\'<div class="metric-box">
        <div class="metric-value">{predicted_score}</div>
        <div class="metric-label">Predicted Final Band Score</div>
        </div>\'\'\',
        unsafe_allow_html=True
    )
with col3:
    st.markdown(
        f\'\'\'<div class="metric-box">
        <div class="metric-value">{tasks_done}/{tasks_total}</div>
        <div class="metric-label">Tasks Completed Today</div>
        </div>\'\'\',
        unsafe_allow_html=True
    )

# ── Progress Bar ─────────────────────────────────────────────
st.markdown("<br>", unsafe_allow_html=True)
st.markdown(f"**Progress towards Target Band {target_band}:**")
st.progress(progress_pct / 100)
st.caption(f"{progress_pct}% of your band gap covered based on today\'s activity projection")

# ── Status Message ───────────────────────────────────────────
if gap <= 0:
    st.markdown(
        \'<div class="on-track">🎉 Congratulations! You have already achieved your target band score. Keep practising to maintain your performance!</div>\',
        unsafe_allow_html=True
    )
elif predicted_score >= target_band:
    st.markdown(
        f\'<div class="on-track">✅ You are ON TRACK! If you maintain this daily effort for {days_left} days, you are projected to reach Band {predicted_score} — meeting your target of Band {target_band}. Keep going! 💪</div>\',
        unsafe_allow_html=True
    )
elif predicted_score >= target_band - 0.5:
    st.markdown(
        f\'<div class="off-track">⚠️ You are CLOSE but not fully on track. Projected score: Band {predicted_score} vs Target: Band {target_band}. Consider increasing daily study time or completing all tasks consistently. Half a band more is very achievable!</div>\',
        unsafe_allow_html=True
    )
else:
    st.markdown(
        f\'<div class="danger">🔴 You are currently OFF TRACK. Projected score: Band {predicted_score} vs Target: Band {target_band}. You need to increase study intensity. Try completing all tasks daily and adding extra focused practice sessions.</div>\',
        unsafe_allow_html=True
    )

# ── Motivational Footer ──────────────────────────────────────
st.markdown("<br>", unsafe_allow_html=True)
st.markdown("---")
st.markdown(
    "<div style=\'text-align:center; color:#4a7ab5; font-size:0.9rem;\'>"
    "🌟 <em>Consistency beats intensity. Show up every day and your target band is within reach.</em>"
    "</div>",
    unsafe_allow_html=True
)
st.markdown(
    "<div style=\'text-align:center; color:#aaa; font-size:0.78rem; margin-top:6px;\'>"
    "IELTS Momentum Engine · Built for focused, data-driven IELTS preparation"
    "</div>",
    unsafe_allow_html=True
)
'''

with open("app.py", "w") as f:
    f.write(app_code)

print("✅ app.py written successfully!")

In [ ]:
# ============================================================
# CELL 3 — Launch Streamlit with ngrok Public URL
# ============================================================
import subprocess
import time
from pyngrok import ngrok, conf

# ── Optional: Set your ngrok authtoken for longer sessions ───
# Sign up free at https://dashboard.ngrok.com/ to get a token
# Uncomment the next line and paste your token:
# ngrok.set_auth_token("YOUR_NGROK_AUTH_TOKEN_HERE")

# ── Kill any existing Streamlit processes ────────────────────
!pkill -f streamlit 2>/dev/null || true
time.sleep(1)

# ── Start Streamlit on port 8501 in background ───────────────
streamlit_proc = subprocess.Popen(
    ["streamlit", "run", "app.py",
     "--server.port=8501",
     "--server.headless=true",
     "--server.enableCORS=false",
     "--server.enableXsrfProtection=false"],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE
)

# Give Streamlit a moment to start
time.sleep(4)

# ── Open ngrok tunnel ────────────────────────────────────────
public_url = ngrok.connect(8501)
print("\n" + "="*55)
print("🚀  IELTS Momentum Engine is LIVE!")
print("="*55)
print(f"  🌐  Public URL: {public_url}")
print("="*55)
print("  • Open the URL above in ANY browser")
print("  • Adjust your profile in the left sidebar")
print("  • Tick task checkboxes to see live predictions")
print("  • Press [ ■ Stop ] in Colab to shut down")
print("="*55)

In [ ]:
# ============================================================
# CELL 4 (Optional) — Graceful Shutdown
# Run this cell when you want to stop the app
# ============================================================
ngrok.kill()
streamlit_proc.terminate()
print("✅ App and ngrok tunnel shut down cleanly.")